In [1]:
# Import Packages
import sys, time

In [10]:
# Include PATHS
DEEP_TIMING_HOME = "/brazos/roysam/hlu8/projects/Deep-TIMING/"

sys.path.append(DEEP_TIMING_HOME + 'DT1-preprocessor/')

sys.path.append(DEEP_TIMING_HOME + 'DT2-detector/Well/')
sys.path.append(DEEP_TIMING_HOME + 'DT2-detector/Well/faster-rcnn/')

sys.path.append(DEEP_TIMING_HOME + 'DT2-detector/Cell')

sys.path.append(DEEP_TIMING_HOME + 'DT3-tracker/')

sys.path.append(DEEP_TIMING_HOME + 'DT4-feature/')

In [20]:
# STEP 0: Specify Parameters for experiments
CORES = 20

# DATASET = "20161215_MM_02_MotileTar"

# DATASET = "20161216_MM_04_ControlTar"

# DATASET = "20170330_MM_02_Nalm6"

DATASET = "20180720_RR_CART_NALM6_TIFF_Block1"

RAW_INPUT_PATH = "/brazos/roysam/hlu8/projects/Deep-TIMING/Data/raw/"
OUTPUT_PATH = "/brazos/roysam/hlu8/projects/Deep-TIMING/Data/results/"

microscope = 'zeiss'
channel_index_dict = {"c1_ORG":"CH0", "c2_ORG":"CH3", "c3_ORG":"CH1", "c4_ORG":"CH2"}
channel_name_dict = {"c1_ORG":"phase_contrast", "c2_ORG":"death", "c3_ORG":"effectors", "c4_ORG":"targets"}



# Input and Output Data Type:
Input_Type = "uint16" # or "uint8"
Output_Type = {"uint8": ['c1_ORG', 'c2_ORG', 'c3_ORG', 'c4_ORG'], "uint16":['c2_ORG',]}
#Output_Type = {"uint8": ['c3_ORG',], "uint16":['c3_ORG',]}

# GAMMA = ['c2_ORG']
GAMMA = []

BLOCKS = ['B'+str(i).zfill(3) for i in range(1,2)]
MAX_NANOWELL_PER_BLOCK = 36
FRAMES = 73
Block_Size = 2048
Nanowell_Size = 281




# A. Preprocessing Pipeline
UMX_Channel = [['c3_ORG','c2_ORG', 0.95], ['c4_ORG', 'c3_ORG', 0.2]]   # Args0 - Args1 * Args2
BKG_Channel = []
# ENHANCE_Channel = ['c2_ORG', 'c3_ORG', 'c4_ORG']

# B. Cell Detector Config
Channel_Mix = ['CH0','CH1','CH2'] # Mix Channels for cell detection, e.g. CH0+CH1+CH2
Cell_Detector_Type = 'FRCNN-Fast' # Other Options include [1]'FRCNN-Slow' or [2]'SSD'

# C. Cell Tracker Config
Cell_Tracker_Type = 'EZ' # Other options include [1]'EZ'

# D. Feature Calculation Config
Effector_Max_Number = 4
Target_Max_Number = 4
Effector_Feature_List = ['x', 'y', 'AR', 'SPEED', 'DM']
Target_Feature_List = ['x', 'y', 'AR', 'SPEED', 'CR', 'DM']
CNN_LSTM_DM = 'True'


In [21]:
# Initialize the output folders
from DT_Init import *

t1 = time.time()

DATASET_PATH = OUTPUT_PATH + DATASET + '/'

DT_Initializer(DATASET_PATH, BLOCKS, CORES)

print("Initialization Time Cost: " + str(time.time() - t1 ))

Initialization Time Cost: 2.139104127883911


In [19]:
# from DT_Init import *

# DATASET_PATH = OUTPUT_PATH + DATASET + '/'

# DT_Reset(DATASET_PATH)

In [22]:
# STEP 1: Detect Nanowells
# Get the sample CH0 Image from each Block, Convert it to 8 bit
from generate_CH0_Sample import *

t1 = time.time()

generate_CH0_samples_master(RAW_INPUT_PATH, OUTPUT_PATH, DATASET, BLOCKS, microscope, CORES)

print("CH0 Sample Preparation Time Cost: " + str(time.time() - t1 ))



CH0 Sample Preparation Time Cost: 1.1994128227233887


In [23]:
# Run Nanowell Detection with each Image, save the results
from nanowell_detector import detect_nanowells

t1 = time.time()

PATH_TO_CKPT=DEEP_TIMING_HOME + 'DT2-detector/Well/faster-rcnn/experiment2/models/frozen_inference_graph.pb'
PATH_TO_LABELS=DEEP_TIMING_HOME + 'DT2-detector/Well/faster-rcnn/experiment2/data/TIMING_nanowell_detection.pbtxt'
PATH_TO_OUTPUT_DIR = OUTPUT_PATH + DATASET + '/'

detect_nanowells(PATH_TO_CKPT, PATH_TO_LABELS, PATH_TO_OUTPUT_DIR, BLOCKS, Nanowell_Number=36, BLOCK_SIZE=2048)

print("CH0 Sample Preparation Time Cost: " + str(time.time() - t1 ))

processing B001 ......
CH0 Sample Preparation Time Cost: 17.91649103164673


In [24]:
# STEP 2.1: Preprocessing Steps if necessary(umx, bg, enhance)
from DT_Preprocessor import *

t1 = time.time()

DT_UNMIX(DEEP_TIMING_HOME, RAW_INPUT_PATH, OUTPUT_PATH, DATASET, BLOCKS, FRAMES, UMX_Channel, CORES)

print("STEP-2 UNMIXING TIME COST: " + str(time.time() - t1))

UNMIXING B001 ......
STEP-2 UNMIXING TIME COST: 22.54496169090271


In [ ]:
# STEP 2.2: ENHANCE Steps
# min_clip_value = 315
# max_clip_value = 1000
# min_pixel_value = 0
# max_pixel_value = 2000


# ENHANCE_Channel = ["c2_ORG"]
# ENHANCE_Parameter = [min_pixel_value, max_pixel_value, min_clip_value, max_clip_value]

# from DT_Preprocessor import *

# t1 = time.time()

# DT_CLIP_ENHANCE(DEEP_TIMING_HOME, RAW_INPUT_PATH, OUTPUT_PATH, DATASET, BLOCKS, FRAMES, ENHANCE_Channel, ENHANCE_Parameter,CORES)

# print("STEP-2.2 ENHANCE TIME COST: " + str(time.time() - t1))

In [25]:
# STEP 3: Crop the images, put them in Bxxx/images/crop_8bit_s or Bxxx/images/crop_16bit_s
from nanowell_cropper import *

t1 = time.time()

CLIP_ARGS = {"c1_ORG":[2000, 20000],"c2_ORG":[160, 180],"c3_ORG":[100, 800],"c4_ORG":[250, 1000]}

DT_CROP_IMAGES(RAW_INPUT_PATH, OUTPUT_PATH, DATASET, BLOCKS, FRAMES, Output_Type, channel_index_dict, CORES, Nanowell_Size, Block_Size, CLIP_ARGS, GAMMA)

print("STEP-3 IMAGE CROP TIME COST: " + str(time.time() - t1))

CROPPING BLOCK B001 ......
STEP-3 IMAGE CROP TIME COST: 39.27587175369263


In [26]:
# STEP 4.1: Cell Detection, including Channel mixing, cell detection, bboxes cleaning
from DT_Cell_Detector import *

CH_MIX = ['CH0', 'CH1', 'CH2']
Detector_Type = 'FRCNN-Fast'
MAXIMUM_CELL_DETECTED = 10

t1 = time.time()

DT_Cell_Detector(DEEP_TIMING_HOME, OUTPUT_PATH, DATASET, BLOCKS, FRAMES, CH_MIX, Detector_Type, MAXIMUM_CELL_DETECTED, Nanowell_Size, CORES)

print("STEP-4 CELL DETECTION TIME COST: " + str(time.time() - t1))

Detecting Cells in B001 ...... 
STEP-4 CELL DETECTION TIME COST: 73.72362565994263


In [27]:
# STEP 4.2: Cell Detection Confinement Constraint
from DT_Cell_Detection_Cleaner import *

t1 = time.time()

CC_THRESHOLD = 0.70
SCORE_THRESHOLD = 0.70
MAX_E_COUNT = 4
MAX_T_COUNT = 4
Detector_Type = 'FRCNN-Fast'

DT_Cell_Cleaner(OUTPUT_PATH, DATASET, BLOCKS, FRAMES, CC_THRESHOLD, SCORE_THRESHOLD, MAX_E_COUNT, MAX_T_COUNT, Detector_Type, CORES)

print("STEP-4.2 CELL DETECTION CLEANING TIME COST: " + str(time.time() - t1))

Applying Confinement Constraint B001 ...... 
STEP-4.2 CELL DETECTION CLEANING TIME COST: 99.86983156204224


In [28]:
# # STEP 5.2: Cell Tracking, options for EZ_track and SIAMESE_track
from DT_EZ_Tracker import *

Cell_Tracker_Type = 'EZ'

DETECTOR_TYPE = Cell_Detector_Type
TRACKER_TYPE = Cell_Tracker_Type

t1 = time.time()

if Cell_Tracker_Type == 'EZ':
    for BLOCK in BLOCKS:
        print('CELL TRACKING for ' + BLOCK + '......')
        DT_EZ_TRACKER(OUTPUT_PATH, DATASET, BLOCK, FRAMES, DETECTOR_TYPE, TRACKER_TYPE, CORES)
    
print("STEP-5 CELL TRACKING TIME COST: " + str(time.time() - t1))    

CELL TRACKING for B001......
STEP-5 CELL TRACKING TIME COST: 3.697517156600952


In [29]:
# STEP 6: Feature Calculation, including CNN-LSTM Model/Apoptosis Intensity calculation using CH3

from DT_FEATURE_WIZARD import *

DETECTOR_TYPE = Cell_Detector_Type
TRACKER_TYPE = Cell_Tracker_Type

Effector_Max_Number = 3
Target_Max_Number = 3
Effector_Feature_List = ['x', 'y', 'AR', 'SPEED', 'DM']
Target_Feature_List = ['x', 'y', 'AR', 'SPEED', 'CR', 'DM']

PARAMETER = [Effector_Max_Number, Effector_Feature_List, Target_Max_Number, Target_Feature_List]

t1 = time.time()

for BLOCK in BLOCKS:
    print('FEATURE CALCULATION for ' + BLOCK + '......')
    DT_FEATURE_EXTRACTOR(OUTPUT_PATH, DATASET, BLOCK, FRAMES, DETECTOR_TYPE, TRACKER_TYPE, PARAMETER, CORES)
    
generate_combined_feat_table(OUTPUT_PATH, DATASET, BLOCKS, FRAMES, DETECTOR_TYPE)
    
print("STEP-6 CELL FEATURE CALCULATION TIME COST: " + str(time.time() - t1))   

FEATURE CALCULATION for B001......


/brazos/roysam/hlu8/anaconda2/envs/tensorflow-1.10/lib/python3.6/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/brazos/roysam/hlu8/anaconda2/envs/tensorflow-1.10/lib/python3.6/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


ASSEMBLE ALL THE FEATURES......
BLOCK : B001 NANOWELL: 15
ADDING lines: 49
BLOCK : B001 NANOWELL: 3
ADDING lines: 49
BLOCK : B001 NANOWELL: 5
ADDING lines: 49
BLOCK : B001 NANOWELL: 1
ADDING lines: 49
BLOCK : B001 NANOWELL: 8
ADDING lines: 49
BLOCK : B001 NANOWELL: 6
ADDING lines: 49
BLOCK : B001 NANOWELL: 14
ADDING lines: 49
BLOCK : B001 NANOWELL: 7
ADDING lines: 49
BLOCK : B001 NANOWELL: 11
ADDING lines: 49
BLOCK : B001 NANOWELL: 9
ADDING lines: 49
BLOCK : B001 NANOWELL: 2
ADDING lines: 49
BLOCK : B001 NANOWELL: 16
ADDING lines: 49
BLOCK : B001 NANOWELL: 18
ADDING lines: 49
BLOCK : B001 NANOWELL: 20
ADDING lines: 49
BLOCK : B001 NANOWELL: 21
ADDING lines: 49
BLOCK : B001 NANOWELL: 22
ADDING lines: 49
BLOCK : B001 NANOWELL: 23
ADDING lines: 49
BLOCK : B001 NANOWELL: 25
ADDING lines: 49
BLOCK : B001 NANOWELL: 26
ADDING lines: 49
BLOCK : B001 NANOWELL: 27
ADDING lines: 49
BLOCK : B001 NANOWELL: 28
ADDING lines: 49
STEP-6 CELL FEATURE CALCULATION TIME COST: 4.500833749771118
